# 0.0 General Section

## Imports

In [1]:
import pandas  as pd
import numpy   as np
import xgboost as xgb
import datetime

from sklearn.model_selection import train_test_split 
from sklearn.preprocessing   import RobustScaler, MinMaxScaler
from sklearn.metrics         import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.ensemble        import RandomForestRegressor

from lightgbm                import LGBMRegressor

#from sklearn.preprocessing import LabelEncoder
#from sklearn.linear_model  import LinearRegression, Lasso



## Support Functions

## Load Data

In [2]:
def load_data(dataset):
    stores  = pd.read_csv( '../data/stores.csv' )
    
    if dataset == 'train':
        train   = pd.read_csv( '../data/train.csv' )
        train_f = pd.read_csv( '../data/train_features.csv' )    
             
        df = pd.merge( train, train_f, on=['loja', 'data', 'feriado'], how='left' ).merge( stores, on='loja', how='left' )
    else:
        test   = pd.read_csv( '../data/test.csv' )
        test_f = pd.read_csv( '../data/test_features.csv' )
             
        df = pd.merge( test, test_f, on=['loja', 'data', 'feriado'], how='left' ).merge( stores, on='loja', how='left' )
        

    return df
    

In [3]:
df_raw = load_data('train')
df_raw.head()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,tipo,tamanho
0,140679,17,93,01-07,6283.00,nao,-14.316667,2.891,NaN,NaN,NaN,NaN,NaN,0.06866,NaN,541,eletronico,93188.0
1,140680,12,17,01-07,10006.77,nao,3.133333,3.287,NaN,NaN,NaN,NaN,NaN,0.14021,NaN,463,eletronico,112238.0
2,140681,3,25,01-07,2658.57,nao,11.861111,NaN,NaN,NaN,NaN,NaN,NaN,0.07551,NaN,901,eletronico,NaN
3,140682,42,1,01-07,8836.00,nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08744,NaN,176,outro,39690.0
4,140683,26,9,01-07,9526.27,nao,-6.038889,3.193,NaN,NaN,NaN,NaN,NaN,0.07907,NaN,124,eletrodomestico,152513.0


In [4]:
df_raw.head()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,tipo,tamanho
0,140679,17,93,01-07,6283.00,nao,-14.316667,2.891,NaN,NaN,NaN,NaN,NaN,0.06866,NaN,541,eletronico,93188.0
1,140680,12,17,01-07,10006.77,nao,3.133333,3.287,NaN,NaN,NaN,NaN,NaN,0.14021,NaN,463,eletronico,112238.0
2,140681,3,25,01-07,2658.57,nao,11.861111,NaN,NaN,NaN,NaN,NaN,NaN,0.07551,NaN,901,eletronico,NaN
3,140682,42,1,01-07,8836.00,nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08744,NaN,176,outro,39690.0
4,140683,26,9,01-07,9526.27,nao,-6.038889,3.193,NaN,NaN,NaN,NaN,NaN,0.07907,NaN,124,eletrodomestico,152513.0


# 1.0 Data Description

In [5]:
df1 = df_raw.copy()

## 1.1 Data Dimensions

In [6]:
print(f'Number of Rows: { df1.shape[0] }' )
print(f'Number of Cols: { df1.shape[1] }' )

Number of Rows: 135385
Number of Cols: 18


## 1.2 List Data Types

In [7]:
# list data types
df1.dtypes

id                          int64
loja                        int64
setor                       int64
data                       object
vendas_semanais           float64
feriado                    object
temperatura               float64
combustivel               float64
desconto_1                float64
desconto_2                float64
desconto_3                float64
desconto_4                float64
desconto_5                float64
desemprego                float64
distancia_competidores    float64
clientes                    int64
tipo                       object
tamanho                   float64
dtype: object

In [8]:
# 1.3 

# 2.0 Data Cleaning

In [9]:
df2 = df1.copy()

# 2.1 Check N/A

In [10]:
df2.isna().sum()

id                             0
loja                           0
setor                          0
data                           0
vendas_semanais            12862
feriado                        0
temperatura                47945
combustivel                52989
desconto_1                129459
desconto_2                129510
desconto_3                129746
desconto_4                130184
desconto_5                129459
desemprego                     0
distancia_competidores    129459
clientes                       0
tipo                           0
tamanho                    33327
dtype: int64

## 2.2 Fillout NA

In [11]:
def data_cleaning( df, dataset ):

    df_temp = df[['loja', 'temperatura']].groupby('loja').mean().reset_index()    
    
    def get_temperatura_media( df, loja ):
        df = df[ df['loja'] == loja].reset_index()
        return df.loc[0, 'temperatura']    

    # descontos
    df['desconto_1'].fillna(0, inplace=True)
    df['desconto_2'].fillna(0, inplace=True)
    df['desconto_3'].fillna(0, inplace=True)
    df['desconto_4'].fillna(0, inplace=True)
    df['desconto_5'].fillna(0, inplace=True)

    # temperatura, tamanho, combustivel
    df['temperatura'] = ( df.apply( lambda x : get_temperatura_media( df_temp, x['loja'] ) if np.isnan( x['temperatura'] ) 
                                                                                         else x['temperatura'], axis=1) )   
    #df['temperatura'].fillna( df['temperatura'].mean(), inplace=True)
    df['tamanho'].fillna( df['tamanho'].mean(), inplace=True)
    df['combustivel'].fillna( df['combustivel'].mean(), inplace=True)

    # distancia_competidores
    df['distancia_competidores'].fillna( 200000, inplace=True )

    # change data types
    df['data'] = pd.to_datetime( df['data'], format='%m-%d' )

    feriados = { 'nao' : 0, 'sim' : 1 }
    df['feriado'] = df['feriado'].map( feriados )    

    # remove lines
    if dataset == 'train':
        df = df[ ~df['vendas_semanais'].isna()]
        df = df[ df['vendas_semanais'] > 0]

    return df


In [12]:
df2 = data_cleaning( df1, 'train' )

In [13]:
df2.head()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,desconto_3,desconto_4,desconto_5,desemprego,distancia_competidores,clientes,tipo,tamanho
0,140679,17,93,1900-01-07,6283.00,0,-14.316667,2.891000,0.0,0.0,0.0,0.0,0.0,0.06866,200000.0,541,eletronico,93188.00000
1,140680,12,17,1900-01-07,10006.77,0,3.133333,3.287000,0.0,0.0,0.0,0.0,0.0,0.14021,200000.0,463,eletronico,112238.00000
2,140681,3,25,1900-01-07,2658.57,0,11.861111,3.597816,0.0,0.0,0.0,0.0,0.0,0.07551,200000.0,901,eletronico,132619.11031
3,140682,42,1,1900-01-07,8836.00,0,21.900831,3.597816,0.0,0.0,0.0,0.0,0.0,0.08744,200000.0,176,outro,39690.00000
4,140683,26,9,1900-01-07,9526.27,0,-6.038889,3.193000,0.0,0.0,0.0,0.0,0.0,0.07907,200000.0,124,eletrodomestico,152513.00000


# Feature Engineering

In [14]:
def feature_engineering( df, dataset ):
    
    df['dia'] = df['data'].dt.day
    df['mes'] = df['data'].dt.month
    df['dia_semana'] = df['data'].dt.dayofweek

    #df['descontos'] = df['desconto_1'] + df['desconto_2'] + df['desconto_3'] + df['desconto_4'] + df['desconto_5']

    # week of year
    df['semana_ano'] = df.apply( lambda x: datetime.date( 2023, x['mes'], x['dia'] ).isocalendar().week, axis=1 )
    

    df= pd.get_dummies( df, prefix=['tipo'], columns=['tipo'] )  

    booleanos = { False : 0, True: 1 }
    df['tipo_eletrodomestico'] = df['tipo_eletrodomestico'].map( booleanos )
    df['tipo_eletronico']      = df['tipo_eletronico'].map( booleanos )
    df['tipo_outro']           = df['tipo_outro'].map( booleanos )

    return df

In [15]:
df3 = feature_engineering( df2, 'train' )

In [16]:
df3.head()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,...,distancia_competidores,clientes,tamanho,dia,mes,dia_semana,semana_ano,tipo_eletrodomestico,tipo_eletronico,tipo_outro
0,140679,17,93,1900-01-07,6283.00,0,-14.316667,2.891000,0.0,0.0,...,200000.0,541,93188.00000,7,1,6,1,0,1,0
1,140680,12,17,1900-01-07,10006.77,0,3.133333,3.287000,0.0,0.0,...,200000.0,463,112238.00000,7,1,6,1,0,1,0
2,140681,3,25,1900-01-07,2658.57,0,11.861111,3.597816,0.0,0.0,...,200000.0,901,132619.11031,7,1,6,1,0,1,0
3,140682,42,1,1900-01-07,8836.00,0,21.900831,3.597816,0.0,0.0,...,200000.0,176,39690.00000,7,1,6,1,0,0,1
4,140683,26,9,1900-01-07,9526.27,0,-6.038889,3.193000,0.0,0.0,...,200000.0,124,152513.00000,7,1,6,1,1,0,0


In [17]:
df3.describe()

,id,loja,setor,data,vendas_semanais,feriado,temperatura,combustivel,desconto_1,desconto_2,...,distancia_competidores,clientes,tamanho,dia,mes,dia_semana,semana_ano,tipo_eletrodomestico,tipo_eletronico,tipo_outro
count,122142.000000,122142.000000,122142.000000,122142,122142.000000,122142.000000,122142.000000,122142.000000,122142.000000,122142.000000,...,122142.000000,122142.000000,122142.000000,122142.00000,122142.000000,122142.0,122142.000000,122142.000000,122142.000000,122142.000000
mean,208333.101447,22.233474,44.091705,1900-06-13 13:51:14.775261696,63188.052121,0.043769,15.713131,3.597514,389.837856,226.273826,...,192000.394975,508.972139,132540.247627,15.25016,5.936926,6.0,23.511036,0.511593,0.386648,0.101759
min,140679.000000,1.000000,1.000000,1900-01-07 00:00:00,0.010000,0.000000,-18.922222,2.891000,0.000000,0.000000,...,1576.000000,1.000000,34875.000000,1.00000,1.000000,6.0,1.000000,0.000000,0.000000,0.000000
25%,174450.250000,11.000000,18.000000,1900-03-25 00:00:00,2268.622500,0.000000,10.311111,3.566000,0.000000,0.000000,...,200000.000000,260.000000,114533.000000,8.00000,3.000000,6.0,12.000000,0.000000,0.000000,0.000000
50%,208342.500000,22.000000,37.000000,1900-06-17 00:00:00,8286.970000,0.000000,16.272222,3.597816,0.000000,0.000000,...,200000.000000,517.000000,132619.110310,15.00000,6.000000,6.0,24.000000,1.000000,0.000000,0.000000
75%,242190.500000,33.000000,72.000000,1900-09-02 00:00:00,23699.117500,0.000000,21.983113,3.682000,0.000000,0.000000,...,200000.000000,761.000000,155083.000000,22.00000,9.000000,6.0,35.000000,1.000000,1.000000,0.000000
max,276063.000000,45.000000,99.000000,1900-11-18 00:00:00,999967.424022,1.000000,37.588889,4.211000,34348.140000,44021.610000,...,200000.000000,999.000000,219622.000000,30.00000,11.000000,6.0,46.000000,1.000000,1.000000,1.000000
std,39100.845536,12.785405,30.459726,NaN,198723.264467,0.204581,9.325959,0.223189,2469.220594,1931.281626,...,37471.184584,290.315519,49611.742648,8.60680,3.055449,0.0,13.285818,0.499868,0.486984,0.302332


## Data Preparation

In [18]:
def data_preparation( df, dataset ):
    if dataset == 'train':
        df['vendas_semanais'] = round( df['vendas_semanais'], 2)
        df['vendas_semanais'] = np.log1p( df['vendas_semanais'] ) 
    
    mms = MinMaxScaler()
    df['loja']         = mms.fit_transform( df[['loja']].values )
    df['setor']        = mms.fit_transform( df[['setor']].values )
    df['temperatura']  = mms.fit_transform( df[['temperatura']].values )
    df['combustivel']  = mms.fit_transform( df[['combustivel']].values )
    df['desconto_1']   = mms.fit_transform( df[['desconto_1']].values )
    df['desconto_2']   = mms.fit_transform( df[['desconto_2']].values )
    df['desconto_3']   = mms.fit_transform( df[['desconto_3']].values )
    df['desconto_4']   = mms.fit_transform( df[['desconto_4']].values )
    df['desconto_5']   = mms.fit_transform( df[['desconto_5']].values )

    rs = RobustScaler()
    df['distancia_competidores'] = rs.fit_transform( df[['distancia_competidores']].values )
    df['clientes']               = rs.fit_transform( df[['clientes']].values )
    df['tamanho']                = rs.fit_transform( df[['tamanho']].values )
    

    df['dia_sin'] = df['dia'].apply( lambda x: np.sin( x * ( 2 * np.pi/30 ) ) )
    df['dia_cos'] = df['dia'].apply( lambda x: np.cos( x * ( 2 * np.pi/30 ) ) )
    
    df['mes_sin'] = df['mes'].apply( lambda x: np.sin( x * ( 2 * np.pi/12 ) ) )
    df['mes_cos'] = df['mes'].apply( lambda x: np.cos( x * ( 2 * np.pi/12 ) ) )
    
    df['dia_semana_sin'] = df['dia_semana'].apply( lambda x: np.sin( x * ( 2 * np.pi/7 ) ) )
    df['dia_semana_cos'] = df['dia_semana'].apply( lambda x: np.cos( x * ( 2 * np.pi/7 ) ) )

    df['semana_ano_sin'] = df['semana_ano'].apply( lambda x: np.sin( x * ( 2 * np.pi/52 ) ) )
    df['semana_ano_cos'] = df['semana_ano'].apply( lambda x: np.cos( x * ( 2 * np.pi/52 ) ) )        

    return df

In [19]:
df4 = data_preparation( df3, 'train' )

## Features Selection

In [20]:
### model definition
#forest = RandomForestRegressor( n_estimators=250, random_state=0, n_jobs=-1 )

### data preparation
#x_train_n = df4.drop( columns=['id', 'data', 'vendas_semanais', 'semana_ano'], axis=1 )
#y_train_n = df4[['vendas_semanais']].values
#forest.fit( x_train_n, y_train_n )

#importances = forest.feature_importances_
#std = np.std( [tree.feature_importances_ for tree in forest.estimators_], axis=0 )
#indices = np.argsort( importances )[::-1]

### print feature ranking
#print('Feature ranking:')
#df_rank = pd.DataFrame()

#for i,j in zip( x_train_n, forest.feature_importances_ ):
#    aux = pd.DataFrame( {'feature' : i, 'importance' : j }, index=[0] )
#    df_rank = pd.concat( [df_rank, aux], axis=0 )

#print( df_rank.sort_values( 'importance', ascending=False ) )

In [38]:
#cols_selected = [ 'loja', 'setor', 'feriado', 'temperatura', 'combustivel', 'desconto_1', 'desconto_2', 'desconto_3',
#                  'desconto_4', 'desconto_5', 'desemprego', 'distancia_competidores', 'clientes', 'tamanho', 
#                  'dia_sin', 'dia_cos', 'mes_sin', 'mes_cos', 'dia_semana_sin', 'dia_semana_cos', 
#                  'tipo_eletrodomestico', 'tipo_eletronico', 'tipo_outro' ] 

cols_selected = ['id', 'vendas_semanais', 'semana_ano', 'setor', 'tamanho', 'clientes', 'temperatura', 'loja', 'desemprego', 'combustivel',
                 'tipo_outro', 'tipo_eletrodomestico', 'semana_ano_sin', 'semana_ano_cos']

In [22]:
# split train in train and test datasets (80/20)
#X_train, X_test, y_train, y_test = train_test_split( df4, 
#                                                     df4['vendas_semanais'].values, 
#                                                     test_size=0.2, 
#                                                     random_state=1 )




In [39]:
X_train = df4.copy()
x_train = X_train[cols_selected]

# Machine Learning

In [24]:
def get_metrics( model, y_real, y_pred ):
    mae  = mean_absolute_error( y_real, y_pred )
    mse  = mean_squared_error( y_real, y_pred )
    rmse = np.sqrt( mean_squared_error( y_real, y_pred ) )

    return pd.DataFrame( { 'Model'      : model,
                           'MAE'        : mae,
                           'RMSE'       : rmse
                           }, index=[0] )

In [25]:
int(46/5)

9

In [26]:
def cross_validation( x_training, kfold, model_name, model, verbose=True ):
    mae_list =  []
    rmse_list = []

    weeks = int( x_training['semana_ano'].max() / kfold )

    for k in range( 1, kfold+1 ):
        
        # start and end week 
        start_week = ( (k-1) * weeks ) + 1
        end_week   = k * weeks

        if verbose:
            print(f'\nValidation Block: { k }. Start Week: { start_week } End Week: { end_week }' )

                                                                            
        # filtering dataset
        training   = x_training[ ( x_training['semana_ano'] < start_week  ) & ( x_training['semana_ano'] > end_week ) ]
        validation = x_training[ ( x_training['semana_ano'] >= start_week ) & ( x_training['semana_ano'] <= end_week ) ]

        # training dataset
        xtraining = training.drop( ['id', 'vendas_semanais', 'semana_ano'], axis=1 )
        ytraining = training['vendas_semanais']

        # validation
        xvalidation = validation.drop( ['id', 'vendas_semanais', 'semana_ano'], axis=1 )
        yvalidation = validation['vendas_semanais']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = get_metrics( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )

        # store performance of each kfold iteration
        mae_list.append( m_result['MAE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {  'model'   : model_name,
                            'mae_cv'  : np.round( np.mean( mae_list ),  2 ),
                            'rmse_cv' : np.round( np.mean( rmse_list ), 2 )
                        }, index=[0] )


### XGBoost

In [40]:
# define model with default values of XGBoost's parameter
xgb_model = xgb.XGBRegressor( objective='reg:squarederror', 
                              n_estimators=100,
                              eta=0.3, 
                              max_depth=6,  
                              subsample=1,
                              colsample_bytree=1,
                              min_child_weight=1
                            )


# calculate performance
xgb_model_result = cross_validation( x_train, 5, 'xgb_model',  xgb_model )
xgb_model_result


Validation Block: 1. Start Week: 1 End Week: 9

Validation Block: 2. Start Week: 10 End Week: 18

Validation Block: 3. Start Week: 19 End Week: 27

Validation Block: 4. Start Week: 28 End Week: 36

Validation Block: 5. Start Week: 37 End Week: 45


,model,mae_cv,rmse_cv
0,xgb_model,63181.43,208485.55


## LGBM

In [28]:
# define model 
#lgbm_model = LGBMRegressor( n_estimators=100, n_jobs=-1, random_state=42)


# calculate performance
#lgbm_model_result = cross_validation( x_train, 5, 'lgbm',  lgbm_model )
#lgbm_model_result

In [29]:
def pipeline(dataset):
    df = load_data(dataset)
    df = data_cleaning( df, dataset )
    df = feature_engineering( df, dataset )
    df = data_preparation( df, dataset )

    return df

In [30]:
cols_selected = ['setor', 'tamanho', 'clientes', 'temperatura', 'loja', 'desemprego', 'combustivel',
                 'tipo_outro', 'tipo_eletrodomestico']

In [31]:
X_test = pipeline('test')
x_test = X_test[cols_selected]
x_test.head()

,setor,tamanho,clientes,temperatura,loja,desemprego,combustivel,tipo_outro,tipo_eletrodomestico
0,0.153061,0.000000,-0.904656,0.591113,0.431818,0.07082,0.595166,0,1
1,0.132653,1.280364,-0.077605,0.936549,0.863636,0.07716,0.208459,0,1
2,0.040816,0.564556,-0.718404,0.355636,0.886364,0.04420,0.661631,0,1
3,0.928571,0.000000,-0.039911,0.512448,0.522727,0.08454,0.892749,0,1
4,0.193878,-0.311548,0.000000,0.589385,0.477273,0.07706,0.377632,0,0


In [32]:
X_train = pipeline('train')
y_train = X_train['vendas_semanais']
x_train = X_train[cols_selected]
x_train.head()

,setor,tamanho,clientes,temperatura,loja,desemprego,combustivel,tipo_outro,tipo_eletrodomestico
0,0.938776,-0.972407,0.047904,0.081498,0.363636,0.06866,0.000000,0,0
1,0.163265,-0.502617,-0.107784,0.390287,0.250000,0.14021,0.300000,0,0
2,0.244898,0.000000,0.766467,0.544731,0.045455,0.07551,0.535466,0,0
3,0.000000,-2.291717,-0.680639,0.722390,0.931818,0.08744,0.535466,1,0
4,0.081633,0.490601,-0.784431,0.227979,0.568182,0.07907,0.228788,0,1


In [33]:
# define model with default values of XGBoost's parameter
xgb_model = xgb.XGBRegressor( objective='reg:squarederror', 
                              n_estimators=100,
                              eta=0.3, 
                              max_depth=6,  
                              subsample=1,
                              colsample_bytree=1,
                              min_child_weight=1
                            )



xgb_model.fit( x_train, y_train )

# create predictions
y_pred = xgb_model.predict( x_test )


X_test['vendas_pred'] = np.expm1( y_pred )

In [34]:
X_test.head()

,id,loja,setor,data,feriado,temperatura,combustivel,desconto_1,desconto_2,desconto_3,...,tipo_outro,dia_sin,dia_cos,mes_sin,mes_cos,dia_semana_sin,dia_semana_cos,semana_ano_sin,semana_ano_cos,vendas_pred
0,276064,0.431818,0.153061,1900-11-25,1,0.591113,0.595166,0.029258,0.000765,0.715799,...,0,-0.866025,0.5,-0.5,0.866025,-0.781831,0.62349,-0.568065,0.822984,22185.988281
1,276065,0.863636,0.132653,1900-11-25,1,0.936549,0.208459,0.019532,0.002803,0.544561,...,0,-0.866025,0.5,-0.5,0.866025,-0.781831,0.62349,-0.568065,0.822984,29385.396484
2,276066,0.886364,0.040816,1900-11-25,1,0.355636,0.661631,0.021580,0.003711,0.284986,...,0,-0.866025,0.5,-0.5,0.866025,-0.781831,0.62349,-0.568065,0.822984,24677.990234
3,276067,0.522727,0.928571,1900-11-25,1,0.512448,0.892749,0.224185,0.000640,0.454033,...,0,-0.866025,0.5,-0.5,0.866025,-0.781831,0.62349,-0.568065,0.822984,142700.906250
4,276068,0.477273,0.193878,1900-11-25,1,0.589385,0.377632,0.143761,0.000000,0.494867,...,0,-0.866025,0.5,-0.5,0.866025,-0.781831,0.62349,-0.568065,0.822984,7603.524414


In [35]:
df_submission = X_test[['id', 'vendas_pred']].rename( columns={ 'vendas_pred' : 'vendas_semanais' } )
df_submission.head()

,id,vendas_semanais
0,276064,22185.988281
1,276065,29385.396484
2,276066,24677.990234
3,276067,142700.906250
4,276068,7603.524414


In [36]:
#df_submission.to_csv( '../data/sub01.csv', index=False )